In [1]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'NSE'

# Add `src` to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path
from pathlib import Path
if 'src' not in Path.cwd().parts:
    src_path = str(Path(get_path('purelib')) / '_src.pth')
    with open(src_path, 'w') as f:
        f.write(str(Path.cwd() / 'src\n'))

# Start the Jupyter loop
from ib_insync import util, IB
util.startLoop()

In [2]:
# Set the root
from from_root import from_root
ROOT = from_root()

from utils import Vars
_vars = Vars(MARKET)
PORT = _vars.PORT
PAPER = _vars.PAPER 
OPT_COLS = _vars.OPT_COLS[0]
DATAPATH = ROOT / 'data' / MARKET.lower()

# Build `states`

In [3]:
# Imports
import asyncio

import numpy as np
import pandas as pd
from ib_insync import MarketOrder

from utils import (get_dte, get_open_orders, get_pickle,
                   get_portfolio_with_margins, get_prices_with_ivs,
                   make_ib_contracts, move_column, get_prec, trade_extracts)

pd.options.display.max_columns = None

## Get margins of pf positions

In [4]:
df_pfm = get_portfolio_with_margins(MARKET)

Getting margins:100%|███████████████████████████| 25/25 [00:01<00:00, 13.41it/s]


## Get open orders

In [5]:
# Get open orders
with IB().connect(port=PORT) as ib:
    df_openords = asyncio.run(get_open_orders(ib)).set_index('conId')

## Combine positions and orders

In [18]:
# Combine positions and orders
df_p = df_pfm.reset_index()
df_p.drop(columns=['contract', 'rePnL'], inplace=True)
df_p.rename(columns={'multiplier': 'mult'}, inplace=True)
df_p.insert(0, 'xnType', 'position')

df_o = df_openords.reset_index()
df_o.insert(0, 'xnType', 'order')
df_o.drop(columns=['order'], inplace=True)
df_o.rename(columns={'totalQuantity': 'qty'}, inplace=True)

## Get underlying prices of portfolio option positions

In [19]:
df = pd.concat([df_p, df_o], ignore_index=True)
df = df.sort_values(['symbol', 'expiry', 'xnType'], ascending=[True, True, False])

# get days to expiry
dte = df.expiry.apply(lambda x: get_dte(x, MARKET))
dte[dte < 0] = 0
df = df.assign(dte = dte)

# for orders fill position with quantities
df.position.fillna(df.qty, inplace=True)

# insert `state` column
df.insert(1, 'state', 'tbd')

# get underlying prices
undPrice = np.where(df.secType == 'OPT', np.nan, df.mktPrice)
df.insert(4, 'undPrice', undPrice)
prices_dict = df.dropna(subset='undPrice').set_index('symbol').mktPrice.to_dict()
df.undPrice = df.symbol.map(prices_dict)

# get prices for missing symbols
symbols = set(df[df.undPrice.isnull()].symbol)
und_contracts = make_ib_contracts(symbols, MARKET)
und_prices = asyncio.run(get_prices_with_ivs(port=PORT, input_contracts = und_contracts, desc = "Getting und prices"))

# merge prices dictionaries
prices_dict = prices_dict | und_prices.set_index('symbol').price.to_dict()
df.undPrice = df.symbol.map(prices_dict)

Getting und prices100%|█████████████████████████| 22/22 [00:01<00:00, 16.89it/s]


# Unreaped
### A naked call or put option that doesn't have an open order to reap. [light-yellow]

In [20]:
# Get the reaped options
reap_mask = df.groupby('conId').position.transform(lambda x: sum(x) == 0)
df.loc[reap_mask, 'state'] = 'reaped'

# Make remaining option positions `unreaped`
unreap_state = (df.xnType == 'position') & (df.state != 'reaped') & (df.secType == 'OPT')
df.loc[unreap_state, 'state'] = 'unreaped'

In [21]:
# Generate the unreaped db
df_unreaped = df[df.state == 'unreaped'].sort_values('mktPrice')

### Check when the trades were made

In [22]:
# check when the trades were registered
reppath = f"{MARKET.lower()}_ib_reports.pkl"
REPORTPATH = DATAPATH.parent / 'master' / reppath

# reports = get_pickle(REPORTPATH)
reports = trade_extracts(MARKET=MARKET)
df_report = reports['trades'].sort_values('time', ascending = False)

cond = df_report.symbol.isin(set(df_unreaped.symbol))
df_rep = df_report[cond].groupby('symbol').head(2)

df_rep.expiry = pd.to_datetime(df_rep.expiry, yearfirst=True)

# merge to get order time
merge_fields = ['symbol', 'strike', 'right', 'expiry']
rep_fields = merge_fields + ['qty', 'time', 'code']

df_r = df_unreaped.merge(df_rep[rep_fields], on = merge_fields, suffixes = [None, "_ordered"])
df_r = df_r.rename(columns={'time': 'time_ordered'})
df_reap = move_column(df_r, 'qty_ordered', 13)
df_reap = move_column(df_reap, 'time_ordered', 6)
df_reap.time_ordered = pd.to_datetime(df_reap.time_ordered).dt.date

# determine the action and quantity
action = np.where(df_reap.position < 1, 'BUY', 'SELL')
qty = -df_reap.position
df_reap = df_reap.assign(action = action, qty = qty)

MAXOPTBUYPRICE = _vars.MAXOPTBUYPRICE
PREC = _vars.PREC

# set the minimum buy price for now
xp = df_reap.mktPrice.apply(lambda x: get_prec(x, PREC))
xp[xp > MAXOPTBUYPRICE] = MAXOPTBUYPRICE
df_reap.lmtPrice= xp

# Uncovered
### A (long/short) stock with no covered (call/put) buy orders

In [ ]:
df[['symbol', 'position', 'mult']]

In [ ]:
# Get the covered options
df.groupby('symbol')[['position', 'mult']].prod()

# Unsowed

In [ ]:
# get unds, open orders and portfolio
unds = set(get_pickle(DATAPATH / 'unds.pkl').keys())
df_openorder, df_pf = asyncio.run(get_order_pf(PORT))

In [ ]:
# No orders to sow and no existing positions

options = df_pf.secType == 'OPT'
stocks = df_pf.secType == 'STK'

long = df_pf.position > 0
long_options = long & options
df_pf[long_options]

unsowed = unds - set(df_openorder.symbol) - set(df_pf.symbol)
unsowed

# Orphaned

In [ ]:
# Long calls or puts without any underlying stock position
options = df.secType == 'OPT'
stocks = df.secType == 'STK'
long = df.position >= 1
long_stocks = long & stocks
long_options = long & options
orphaned = long_stocks & long_options

orphaned = df[orphaned]
orphaned